In [ ]:
import json
import requests
import time

### should be outside the repo
cred_location = r'/Users/jj/code/strava_creds/'

In [ ]:
def get_client_tokens():
    print('please go here https://www.strava.com/settings/api, and copy the token information as prompted')
    client_id = input('please enter your client id')
    client_secret = input('please enter your client secret')
    refresh_token = input('please enter your refresh token')

    client_tokens = {
        'client_id':str(client_id),
        'client_secret':str(client_secret),
        'refresh_token':str(refresh_token)
    }

    with open(cred_location+'client_tokens.json', 'w') as outfile:
        json.dump(client_tokens, outfile, indent=4)

    print("client tokens saved")
    return None

### This will just check if the client tokens exist, not if they are valid.
### client token, client secret, and refresh token should all be held in a json file called client_tokens.json
def check_client_tokens():
    try:
        client_tokens = json.load(open(cred_location+'client_tokens.json'))
        if {'client_id', 'client_secret', 'refresh_token'} == set(client_tokens.keys()):
            print('Client Tokens Found')
            return True
        else:
            return False
    except:
        return False

if check_client_tokens() == False:
    get_client_tokens()

In [ ]:
def authorize():

      client_tokens = json.load(open(cred_location+'client_tokens.json'))

      auth_url = "https://www.strava.com/oauth/token"

      ### add some more configuration
      payload = {**client_tokens, 
                 **{
                        'grant_type':'refresh_token',
                        'f':'json'
                  }
            }

      print(payload)

      print("Requesting Token...\n")
      res = requests.post(auth_url, data=payload, verify=False)
      access_token = res.json()

      if res.status_code < 300:
            with open(cred_location+'access_tokens.json', 'w') as outfile:
                  json.dump(access_token, outfile, indent=4)
            print('Authorisation Complete')

      else:
            print('Authorisation Error')
            print(res.json())


def check_authorization():
      try:
            tokens = json.load(open(cred_location+'access_tokens.json'))
            if time.time() < int(tokens['expires_at']) and 'access_token' in tokens.keys():
                  print((int(tokens['expires_at']) - int(time.time())) / 60, 'minutes until token expires')
                  print('Unexpired Token Found')
                  return True
            else:
                  return False
      except:
            return False

if check_authorization() == False:
      authorize()